In [1]:
import os
os.chdir('../streamlit')

FileNotFoundError: [WinError 2] The system cannot find the file specified: '../streamlit'

In [ ]:
from utils import load_all_data

all_data = load_all_data()
